In [22]:
import psutil
import os

In [23]:
process = psutil.Process(os.getpid())
t = process.memory_info()

In [24]:
t.vms, t.rss

(842641408, 88780800)

In [25]:
def mem_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / psutil.virtual_memory().total

In [26]:
mem_usage()

0.005377140343735092

2\. [TQDM](https://github.com/tqdm/tqdm) gives you progress bars.

In [27]:
from time import sleep

In [28]:
# Without TQDM
s = 0
for i in range(10):
    s += i
    sleep(0.2)
print(s)

45


In [29]:
# With TQDM
from tqdm import tqdm

s = 0
for i in tqdm(range(10)):
    s += i
    sleep(0.2)
print(s)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.97it/s]

45


In [30]:
import os, numpy as np, pickle
from bz2 import BZ2File
from datetime import datetime
from pprint import pprint
from time import time
from tqdm import tqdm_notebook
from scipy import sparse

from urllib.request import urlopen

In [31]:
S  = sparse.csr_matrix(np.array([[1,2],[3,4]]))
Sr= S.sum(axis=0).A1
Sr

array([4, 6])

In [32]:
np.take(Sr, S.indices)

array([4, 6, 4, 6])

In [33]:
PATH = 'data/dbpedia/'
URL_BASE = 'http://downloads.dbpedia.org/3.5.1/en/'
filenames = ["redirects_en.nt.bz2", "page_links_en.nt.bz2"]

for filename in filenames:
    if not os.path.exists(PATH+filename):
        print("Downloading '%s', please wait..." % filename)
        open(PATH+filename, 'wb').write(urlopen(URL_BASE+filename).read())


KeyboardInterrupt



In [ ]:
redirects_filename = PATH+filenames[0]
page_links_filename = PATH+filenames[1]

### Graph Adjacency Matrix

One line of the file looks like:
- `<http://dbpedia.org/resource/AfghanistanHistory> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/History_of_Afghanistan> .`

In the below slice, the plus 1, -1 are to remove the <>

In [ ]:
DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

In [ ]:
def get_lines(filename): return (line.split() for line in BZ2File(filename))

Loop through redirections and create dictionary of source to final destination

In [ ]:
def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ

In [ ]:
def get_redirects(redirects_filename):
    redirects={}
    lines = get_lines(redirects_filename)
    return {src[SLICE]:get_redirect(targ[SLICE], redirects) 
                for src,_,targ,_ in tqdm_notebook(lines, leave=False)}

In [ ]:
redirects = get_redirects(redirects_filename)

In [ ]:
mem_usage()

In [ ]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    lst.append(index_map.setdefault(redirects.get(k, k), len(index_map)))

In [ ]:
limit=119077682 #5000000

In [ ]:
# Computing the integer index map
index_map = dict() # links->IDs
lines = get_lines(page_links_filename)
source, destination, data = [],[],[]
for l, split in tqdm_notebook(enumerate(lines), total=limit):
    if l >= limit: break
    add_item(source, redirects, index_map, split[0])
    add_item(destination, redirects, index_map, split[2])
    data.append(1)

In [ ]:
n=len(data); n

In [ ]:
index_map.popitem()

In [ ]:
[i for i,x in enumerate(source) if x == 9991173]

In [ ]:
source[119077649], destination[119077649]

Now, we want to check which page is the source (has index $9991050$).  Note: usually you should not access a dictionary by searching for its values.  This is inefficient and not how dictionaries are intended to be used.

In [ ]:
for page_name, index in index_map.items():
    if index == 9991050:
        print(page_name)

We can see on Wikipedia that the Cincinati Red Teams Issue has [redirected to W711-2](https://en.wikipedia.org/wiki/W711-2):

<img src="images/cincinnati_reds.png" alt="" style="width: 70%"/>

In [ ]:
test_inds = [i for i,x in enumerate(source) if x == 9991050]

In [ ]:
len(test_inds)

In [ ]:
test_inds[:5]

In [ ]:
test_dests = [destination[i] for i in test_inds]

Now, we want to check which page is the source (has index 9991174):

In [ ]:
for page_name, index in index_map.items():
    if index in test_dests:
        print(page_name)

Below we create a sparse matrix using Scipy's COO format, and that convert it to CSR.

**Questions**: What are COO and CSR?  Why would we create it with COO and then convert it right away?

In [ ]:
X = sparse.coo_matrix((data, (destination,source)), shape=(n,n), dtype=np.float32)
X = X.tocsr()

In [ ]:
del(data,destination, source)

In [ ]:
X

In [ ]:
names = {i: name for name, i in index_map.items()}

In [ ]:
mem_usage()

### Save matrix so we don't have to recompute

In [ ]:
pickle.dump(X, open(PATH+'X.pkl', 'wb'))
pickle.dump(index_map, open(PATH+'index_map.pkl', 'wb'))

In [ ]:
X = pickle.load(open(PATH+'X.pkl', 'rb'))
index_map = pickle.load(open(PATH+'index_map.pkl', 'rb'))

In [ ]:
names = {i: name for name, i in index_map.items()}

In [ ]:
X

## Power method

### Motivation

An $n \times n$ matrix $A$ is **diagonalizable** if it has $n$ linearly independent eigenvectors $v_1,\, \ldots v_n$.

Then any $w$ can be expressed $w = \sum_{j=1}^n c_j v_j $, for some scalars $c_j$.

**Exercise:** Show that $$ A^k w = \sum_{j=1}^n c_j \lambda_j^k v_j$$

**Question**: How will this behave for large $k$?

This is inspiration for the **power method**.

### Code

In [ ]:
def show_ex(v):
    print(', '.join(names[i].decode() for i in np.abs(v.squeeze()).argsort()[-1:-10:-1]))

In [ ]:
?np.squeeze

How to normalize a sparse matrix:

In [ ]:
S = sparse.csr_matrix(np.array([[1,2],[3,4]]))
Sr = S.sum(axis=0).A1; Sr

In [ ]:
S.indices

In [ ]:
S.data

In [ ]:
S.data / np.take(Sr, S.indices)

In [ ]:
def power_method(A, max_iter=100):
    n = A.shape[1]
    A = np.copy(A)
    A.data /= np.take(A.sum(axis=0).A1, A.indices)

    scores = np.ones(n, dtype=np.float32) * np.sqrt(A.sum()/(n*n)) # initial guess
    for i in range(max_iter):
        scores = A @ scores
        nrm = np.linalg.norm(scores)
        scores /= nrm
        print(nrm)

    return scores

**Question**: Why normalize the scores on each iteration?

In [ ]:
scores = power_method(X, max_iter=10)

In [ ]:
show_ex(scores)

In [ ]:
mem_usage()

### Comments

### Compare to SVD

In [ ]:
%time U, s, V = randomized_svd(X, 3, n_iter=3)

In [ ]:
mem_usage()

In [ ]:
# Top wikipedia pages according to principal singular vectors
show_ex(U.T[0])

In [ ]:
show_ex(U.T[1])

In [ ]:
show_ex(V[0])

In [ ]:
show_ex(V[1])

## QR Algorithm

We used the power method to find the eigenvector corresponding to the largest eigenvalue of our matrix of Wikipedia links.  This eigenvector gave us the relative importance of each Wikipedia page (like a simplified PageRank).

Next, let's look at a method for finding all eigenvalues of a symmetric, positive definite matrix.  This method includes 2 fundamental algorithms in numerical linear algebra, and is a basis for many more complex methods.

[The Second Eigenvalue of the Google Matrix](https://nlp.stanford.edu/pubs/secondeigenvalue.pdf): has "implications for the convergence rate of the standard PageRank algorithm as the web scales, for the stability of PageRank to perturbations to the link structure of the web, for the detection of Google spammers, and for the design of algorithms to speed up PageRank".


### Avoiding Confusion: QR Algorithm vs QR Decomposition

The **QR algorithm** uses something called the **QR decomposition**.  Both are important, so don't get them confused.  The **QR decomposition** decomposes a matrix $A = QR$ into a set of orthonormal columns $Q$ and a triangular matrix $R$.  We will look at several ways to calculate the QR decomposition in a future lesson.  For now, just know that it is giving us an orthogonal matrix and a triangular matrix.

### Linear Algebra

Two matrices $A$ and $B$ are **similar** if there exists a non-singular matrix $X$ such that $$B = X^{-1}AX$$

Watch this: [Change of Basis](https://www.youtube.com/watch?v=P2LTAUO1TdA&index=13&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)

**Theorem**: If $X$ is non-singular, then $A$ and $X^{-1}AX$ have the same eigenvalues.

#### More Linear Algebra

A **Schur factorization** of a matrix $A$ is a factorization:
$$ A = Q T Q^*$$
where $Q$ is unitary and $T$ is upper-triangular.

**Question**: What can you say about the eigenvalues of $A$?

**Theorem:** Every square matrix has a Schur factorization.

#### Other resources

Review: [Linear combinations, span, and basis vectors](https://www.youtube.com/watch?v=k7RM-ot2NWY&index=3&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)

See Lecture 24 for proofs of above theorems (and more!)

### Algorithm

The most basic version of the QR algorithm:

    for k=1,2,...
        Q, R = A
        A = R @ Q
        
Under suitable assumptions, this algorithm converges to the Schur form of A!

#### Why it works

Written again, only with subscripts:

$A_0 = A$

for $k=1,2,\ldots$

   $\quad Q_k$, $R_k$ = $A_{k-1}$
    
   $\quad A_k$ = $R_k Q_k$
        
We can think of this as constructing sequences of $A_k$, $Q_k$, and $R_k$.

$$ A_k = Q_k \, R_k $$

$$ Q_k^{-1} \, A_k = R_k$$

Thus, 

$$ R_k Q_k = Q_k^{-1} \, A_k \, Q_k $$

$$A_k = Q_k^{-1} Q_2^{-1} Q_1^{-1} A Q_1 Q_2 \dots Q_k$$

Trefethen proves the following on page 216-217:

$$A^k = Q_1 Q_2 \dots Q_k R_k R_{k-1}\dots R_1$$

**Key**: The QR algorithm constructs orthonormal bases for successive powers $A^k$.  And remember the close relationship between powers of A and the eigen decomposition.

To learn more, read up on *Rayleigh quotients*.

#### Pure QR

In [ ]:
import numpy as np
n = 6
A = np.random.rand(n,n)
AT = A @ A.T
A

In [ ]:
def pure_qr(A, max_iter=50000):
    Ak = np.copy(A)
    n = A.shape[0]
    QQ = np.eye(n)
    for k in range(max_iter):
        Q, R = np.linalg.qr(Ak)
        Ak = R @ Q
        QQ = QQ @ Q
        #if k % 100 == 0:
        #    print(Ak)
         #   print("\n")
    return Ak, QQ

In [ ]:
Ak, QQ = pure_qr(A)

In [ ]:
QQ

#### Pure QR

In [ ]:
Ak, Q = pure_qr(A)

Let's compare to the eigenvalues:

In [ ]:
np.linalg.eigvals(A)

Check that Q is orthogonal:

In [ ]:
np.allclose(np.eye(n), Q @ Q.T), np.allclose(np.eye(n), Q.T @ Q)

This is really really slow.

#### Practical QR (QR with shifts)

**Idea**: Instead of factoring $A_k$ as $Q_k R_k$, 

1. Get the QR factorization $$A_k - s_k I = Q_k R_k$$
2. Set $$A_{k+1} = R_k Q_k + s_k I$$

Choose $s_k$ to approximate an eigenvalue of $A$.  We'll use $s_k = A_k(m,m)$. 

The idea of adding shifts to speed up convergence shows up in many algorithms in numerical linear algebra (including the power method, inverse iteration, and Rayleigh quotient iteration).   

#### Homework: Add shifts to the QR algorithm

In [ ]:
#Exercise: Add shifts to the QR algorithm
#Exercise: def practical_qr(A, iters=10):
#Exercise:     return Ak, Q


#### Practical QR

In [ ]:
Ak, Q = practical_qr(A, 10)

Check that Q is orthogonal:

In [ ]:
np.allclose(np.eye(n), Q @ Q.T), np.allclose(np.eye(n), Q.T @ Q)

Let's compare to the eigenvalues:

In [ ]:
np.linalg.eigvals(A)

**Problem**: This is better than the unshifted version (which wasn't even guaranteed to converge), but is still really slow!  In fact, it is $\mathcal{O}(n^4)$, which is awful.

In the case of symmetric matrices, it's $\mathcal{O}(n^3)$

However, if you start with a **Hessenberg matrix** (zeros below the first subdiagonal), it's faster: $\mathcal{O}(n^3)$, and $\mathcal{O}(n^2)$ if symmetric.

## A Two-Phase Approach

In practice, a two phase approach is used to find eigenvalues:

1. Reduce the matrix to *Hessenberg* form (zeros below the first subdiagonal)
2. Iterative process that causes Hessenberg to converge to a *triangular* matrix.  The eigenvalues of a triangular matrix are the values on the diagonal, so we are finished!

<img src="images/nonhermitian_eigen.JPG" alt="2 phase approach" style="width: 80%"/>
(source: Trefethen, Lecture 25)

In the case of a Hermitian matrix, this approach is even faster, since the intermediate step is also Hermitian (and a Hermitian Hessenberg is *tridiagonal*).

<img src="images/hermitian_eigen.JPG" alt="2 phase approach" style="width: 80%"/>
(source: Trefethen, Lecture 25)

Phase 1 reaches an exact solution in a finite number of steps, whereas Phase 2 theoretically never reaches the exact solution.

We've already done step 2: the QR algorithm.  Remember that it would be possible to just use the QR algorithm, but ridiculously slow.

## Arnoldi Iteration

We can use the Arnoldi iteration for phase 1 (and the QR algorithm for phase 2).

#### Initializations

In [ ]:
import numpy as np
n = 5
A0 = np.random.rand(n,n)  #.astype(np.float64)
A = A0 @ A0.T

np.set_printoptions(precision=5, suppress=True)

### Linear Algebra Review: Projections

When vector $\mathbf{b}$ is projected onto a line $\mathbf{a}$, its projection $\mathbf{p}$ is the part of $\mathbf{b}$ along that line $\mathbf{a}$.

Let's look at interactive graphic (3.4) for [section 3.2.2: Projections](http://immersivemath.com/ila/ch03_dotproduct/ch03.html) of the [Immersive Linear Algebra online book](http://immersivemath.com/ila/index.html).

<img src="images/projection_line.png" alt="projection" style="width: 70%"/>
(source: [Immersive Math](http://immersivemath.com/ila/ch03_dotproduct/ch03.html))

And here is what it looks like to project a vector onto a plane:

<img src="images/projection.png" alt="projection" style="width: 70%"/>
(source: [The Linear Algebra View of Least-Squares Regression](https://medium.com/@andrew.chamberlain/the-linear-algebra-view-of-least-squares-regression-f67044b7f39b))

When vector $\mathbf{b}$ is projected onto a line $\mathbf{a}$, its projection $\mathbf{p}$ is the part of $\mathbf{b}$ along that line $\mathbf{a}$.  So $\mathbf{p}$ is some multiple of $\mathbf{a}$. Let $\mathbf{p} = \hat{x}\mathbf{a}$ where $\hat{x}$ is a scalar.

#### Orthogonality

**The key to projection is orthogonality:** The line *from* $\mathbf{b}$ to $\mathbf{p}$ (which can be written $\mathbf{b} - \hat{x}\mathbf{a}$) is perpendicular to $\mathbf{a}$.

This means that $$ \mathbf{a} \cdot (\mathbf{b} -  \hat{x}\mathbf{a}) = 0 $$

and so $$\hat{x} = \frac{\mathbf{a} \cdot \mathbf{b}}{\mathbf{a} \cdot \mathbf{a}} $$

### The Algorithm

**Motivation**:

We want orthonormal columns in $Q$ and a Hessenberg $H$ such that $A Q = Q H$.

Thinking about it iteratively, $$ A Q_n = Q_{n+1} H_n $$ where $Q_{n+1}$ is $n\times n+1$ and $H_n$ is $n+1 \times n$.  This creates a solvable recurrence relation.

<img src="images/arnoldi.jpg" alt="arnoldi" style="width: 95%"/>
(source: Trefethen, Lecture 33)

**Pseudo-code for Arnoldi Algorithm**

    Start with an arbitrary vector (normalized to have norm 1) for first col of Q
    for n=1,2,3...
        v = A @ nth col of Q
        for j=1,...n
            project v onto q_j, and subtract the projection off of v
            want to capture part of v that isn't already spanned by prev columns of Q
            store coefficients in H
        normalize v, and then make it the (n+1)th column of Q

Notice that we are multiplying A by the previous vector in Q and removing the components that are not orthogonal to the existing columns of Q.

**Question:** Repeated multiplications of A?  Does this remind you of anything?

#### Answer:

In [ ]:
#Exercise Answer
The *Power Method* involved iterative multiplications by A as well!  

### About how the Arnoldi Iteration works

- With the Arnoldi Iteration, we are finding an orthonormal basis for the *Krylov subspace*. 
The Krylov matrix $$ K = \left[b \; Ab \; A^2b \; \dots \; A^{n-1}b \right]$$
has a QR factorization
$$K = QR$$
and that is the same $Q$ that is being found in the Arnoldi Iteration.  Note that the Arnoldi Iteration does not explicity calculate $K$ or $R$.

- Intuition: K contains good information about the largest eigenvalues of A, and the QR factorization reveals this information by peeling off one approximate eigenvector at a time.


The Arnoldi Iteration is two things:
1. the basis of many of the iterative algorithms of numerical linear algebra
2. a technique for finding eigenvalues of nonhermitian matrices
(Trefethen, page 257)

**How Arnoldi Locates Eigenvalues**

1. Carry out Arnoldi iteration
2. Periodically calculate the eigenvalues (called *Arnoldi estimates* or *Ritz values*) of the Hessenberg H, using the QR algorithm
3. Check at whether these values are converging.  If they are, they're probably eigenvalues of A.

### Implementation

In [ ]:
# Decompose square matrix A @ Q ~= Q @ H
def arnoldi(A):
    m, n = A.shape
    assert(n <= m)
    
    # Hessenberg matrix
    H = np.zeros([n+1,n]) #, dtype=np.float64)
    # Orthonormal columns
    Q = np.zeros([m,n+1]) #, dtype=np.float64)
    # 1st col of Q is a random column with unit norm
    b = np.random.rand(m)
    Q[:,0] = b / np.linalg.norm(b)
    for j in range(n):
        v = A @ Q[:,j]
        for i in range(j+1):
            #This comes from the formula for projection of v onto q.
            #Since columns q are orthonormal, q dot q = 1
            H[i,j] = np.dot(Q[:,i], v)
            v = v - (H[i,j] * Q[:,i])
        H[j+1,j] = np.linalg.norm(v)
        Q[:,j+1] = v / H[j+1,j]
        
        # printing this to see convergence, would be slow to use in practice
        print(np.linalg.norm(A @ Q[:,:-1] - Q @ H))
    return Q[:,:-1], H[:-1,:]

In [ ]:
Q, H = arnoldi(A)

Check that H is tri-diagonal:

In [ ]:
H

#### Exercise

Write code to confirm that:
1. AQ = QH
2. Q is orthonormal

#### Answer:

In [ ]:
#Exercise:
np.allclose(A @ Q, Q @ H)

In [ ]:
#Exercise:
np.allclose(np.eye(len(Q)), Q.T @ Q)

#### General Case:

**General Matrix**: Now we can do this on our general matrix A (not symmetric).  In this case, we are getting a Hessenberg instead of a Tri-diagonal

In [ ]:
Q0, H0 = arnoldi(A0)

Check that H is Hessenberg:

In [ ]:
H0

In [ ]:
np.allclose(A0 @ Q0, Q0 @ H0)

In [ ]:
np.allclose(np.eye(len(Q0)), Q0.T @ Q0), np.allclose(np.eye(len(Q0)), Q0 @ Q0.T)

## Putting it all together

In [ ]:
def eigen(A, max_iter=20):
    Q, H = arnoldi(A)
    Ak, QQ = practical_qr(H, max_iter)
    U = Q @ QQ
    D = np.diag(Ak)
    return U, D

In [ ]:
n = 10
A0 = np.random.rand(n,n)
A = A0 @ A0.T

In [ ]:
U, D = eigen(A, 40)

In [ ]:
D

In [ ]:
np.linalg.eigvals(A)

In [ ]:
np.linalg.norm(U @ np.diag(D) @ U.T - A)

In [ ]:
np.allclose(U @ np.diag(D) @ U.T, A, atol=1e-3)

### Further Reading

Let's find some eigenvalues!


from [Nonsymmetric Eigenvalue Problems](https://sites.math.washington.edu/~morrow/498_13/eigenvalues.pdf) chapter:

Note that "direct" methods must still iterate, since finding eigenvalues is mathematically equivalent to finding zeros of polynomials, for which no noniterative methods can exist. We call a method direct if experience shows that it (nearly) never fails to converge in a
fixed number of iterations.

Iterative methods typically provide approximations only to a subset of the eigenvalues and eigenvectors and are usually run only long enough to get a few adequately accurate eigenvalues rather than a large number

our ultimate algorithm: the shifted Hessenberg QR algorithm

More reading:
- [The Symmetric Eigenproblem and SVD](https://sites.math.washington.edu/~morrow/498_13/eigenvalues2.pdf)
- [Iterative Methods for Eigenvalue Problems](https://sites.math.washington.edu/~morrow/498_13/eigenvalues3.pdf) Rayleigh-Ritz Method, Lanczos algorithm

### Coming Up

We will be coding our own QR decomposition (two different ways!) in the future, but first we are going to see another way that the QR decomposition can be used: to calculate linear regression.

## End

### Miscellaneous Notes

Symmetric matrices come up naturally:
- distance matrices
- relationship matrices (Facebook or LinkedIn)
- ODEs

We will look at positive definite matrices, since that guarantees that all the eigenvalues are real.

Note: in the confusing language of NLA, the QR algorithm is *direct*, because you are making progress on all columns at once.  In other math/CS language, the QR algorithm is *iterative*, because it iteratively converges and never reaches an exact solution.

structured orthogonalization.  In the language of NLA, Arnoldi iteration is considered an *iterative* algorithm, because you could stop part way and have a few columns completed.

a Gram-Schmidt style iteration for transforming a matrix to Hessenberg form